Double DQN With Project Barry

In [11]:
import numpy as np
from gym.spaces import Space, Box
from poke_env.player import Gen8EnvSinglePlayer, Gen9EnvSinglePlayer
from gym.utils.env_checker import check_env
from poke_env.environment.abstract_battle import AbstractBattle
from poke_env.environment.side_condition import STACKABLE_CONDITIONS, SideCondition
from poke_env.player import RandomPlayer, wrap_for_old_gym_api
from poke_env.player import RandomPlayer
from poke_env.player import background_evaluate_player


import math
import random
from collections import namedtuple, deque
from itertools import count

#Insert in Pytorch Code here
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR

DDQN Model

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
# Get number of actions from gym action space


class DQN(nn.Module):

    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(n_observations, 128)
        self.layer2 = nn.Linear(128, 256)
        self.layer3 = nn.Linear(256, 128)
        self.layer4 = nn.Linear(128, n_actions)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.relu(self.layer3(x))
        return self.layer4(x)

# Battle Bot Env
Things to look into:
HP for own pokemon actual number or %?

In [13]:
battle_tester = None

class Barry(Gen9EnvSinglePlayer):
    def calc_reward(self, last_battle, current_battle) -> float:
        global battle_tester
        battle_tester = current_battle
        if current_battle.won:
            print("Game Won")
        if current_battle.lost:
            print("Game Lost")
        return self.reward_computing_helper(
            current_battle, fainted_value=2.0, hp_value=1.0, victory_value=30.0
        )
    
    def encode_status(self, status):
        if status == None:
            return [0, 0, 0, 0, 0]
        encoded = []
        for s in ("FNT", "BRN", "PSN", "SLP", "TOX"): #PAR and FRZ not included for this bot
            if status.name == s:
                encoded.append(1)
            else:
                encoded.append(0)
        return encoded

    def encode_hazards(self, hazard_dict):
        encoded = [0]
        if hazard_dict.get(SideCondition(16)):
            encoded[0] = 1
        for h in (15, 19):
            if hazard_dict.get(SideCondition(h)) != None:
                encoded.append(hazard_dict.get(SideCondition(h)))
            else:
                encoded.append(0)
        return encoded
    
    def team_preview_embedding(self): 
        battle = []
        #Active Field
        battle += [0]*14
        battle += [0, 0, 0, 0, 0, 100, 1, 0]*6
        battle += battle
        return np.float32(battle) #size 124

    def embed_battle(self, battle: AbstractBattle):
        
        if battle._in_team_preview:
            return self.team_preview_embedding()
        
        active_field = []
        # 0:5 - which pokemon is active
        # 6:10 - Modifiers
        # 11:13 - Hazard
        # print("EMBEDDING BATTLE")
        # print(battle)
        # print(battle._team)
        # print(battle._teampreview)
        # print(battle._teampreview_opponent_team)
        # print(battle._in_team_preview)
        global battle_tester
        battle_tester = battle
        for k in battle.team.keys():
            if battle.team[k].active:
                active_field.append(1)
            else:
                active_field.append(0)
        #active_field[list(battle.team).index("p1: " + battle.active_pokemon.species.capitalize())] = 1
        active_field.append(battle.active_pokemon.boosts.get("atk"))
        active_field.append(battle.active_pokemon.boosts.get("def"))
        active_field.append(battle.active_pokemon.boosts.get("spa"))
        active_field.append(battle.active_pokemon.boosts.get("spd"))
        active_field.append(battle.active_pokemon.boosts.get("spe")) #11
        active_field = active_field + self.encode_hazards(battle.side_conditions) #Len 14

        pokemon_team = [] #np.ones(42)
        # For all 6 Pokemon
        # 0:4 - Status
        # 5 - HP
        # 6 - has_item
        # 7 - Terastralized
        
        for p in list(battle.team):
            poke = battle.team[p]
            poke_info = self.encode_status(poke.status)# Len 5
            poke_info.append(poke.current_hp_fraction*100) # Len 1
            poke_info.append(int(poke.item != "")) # Len 1
            poke_info.append(int(poke.terastallized)) # Len 1
            pokemon_team = pokemon_team + poke_info # Len 48

        opponent_active_field = []
        # 0:5 - which pokemon is active
        # 6:10 - Modifiers
        # 11:13 - Hazard
        for tpo in list(battle._teampreview_opponent_team):
            opponent_active_field.append(int(tpo.species == battle.opponent_active_pokemon.species))
            
        opponent_active_field.append(battle.opponent_active_pokemon.boosts.get("atk"))
        opponent_active_field.append(battle.opponent_active_pokemon.boosts.get("def"))
        opponent_active_field.append(battle.opponent_active_pokemon.boosts.get("spa"))
        opponent_active_field.append(battle.opponent_active_pokemon.boosts.get("spd"))
        opponent_active_field.append(battle.opponent_active_pokemon.boosts.get("spe"))
        opponent_active_field = opponent_active_field + self.encode_hazards(battle.opponent_side_conditions)

        opponent_pokemon_team = []
        # For all 6 Pokemon
        # 0:3 - Status
        # 4 - HP
        # 5 - has_item
        # 6 - Terastralized
        for p in list(battle._teampreview_opponent_team):
            if ("p2: " + p.species.capitalize()) in battle.opponent_team:
                opponent_pokemon = battle.opponent_team["p2: " + p.species.capitalize()]
                op_poke_info = self.encode_status(opponent_pokemon.status)
                op_poke_info.append(opponent_pokemon.current_hp_fraction*100) #Check if needs to be fraction?
                op_poke_info.append(int(opponent_pokemon.item != ""))
                op_poke_info.append(int(opponent_pokemon.terastallized))
            #opponent_pokemon = battle.opponent_team["p2: " + p.species.capitalize()]
            #if opponent_pokemon == None:
            #    op_poke_info = [0, 0, 0, 0, 0, 100, 1, 0]
            else:
                op_poke_info = [0, 0, 0, 0, 0, 100, 1, 0]
                
            opponent_pokemon_team = opponent_pokemon_team + op_poke_info
        # print(len(active_field))
        # print(len(pokemon_team))
        # print(len(opponent_active_field))
        # print(len(opponent_pokemon_team))
        # Final vector
        final_vector = active_field + pokemon_team + opponent_active_field + opponent_pokemon_team
        return np.float32(final_vector)

    def describe_embedding(self) -> Space: #Still need to complete
        low = []
        high = []
        
        #Active Pokemon
        low += [0, 0, 0, 0, 0, 0]
        high += [1, 1, 1, 1, 1, 1]
        
        #Modifier
        low += [0, 0, 0, 0, 0]
        high += [4, 4, 4, 4, 4]

        #Hazards
        low += [0, 0, 0] 
        high += [1, 3, 3] #CHECK SPIKES

        #Each Pokemon
        poke_low = []
        poke_high = []
        #Status
        poke_low += [0, 0, 0, 0, 0]
        poke_high += [1, 1, 1, 1, 1]
        #HP
        poke_low += [0]
        poke_high += [100] 
        #Has_Item
        poke_low += [0]
        poke_high += [1]
        #Terastralized
        poke_low += [0]
        poke_high += [1]
        #Add to embedding
        low += poke_low*6
        high += poke_high*6

        #Repeat for opponent side
        low += low
        high += high
        return Box(
            np.array(low, dtype=np.float32),
            np.array(high, dtype=np.float32),
            dtype=np.float32,
        )

## Teams

In [14]:
SUSHO = """
Volcarona @ Heavy-Duty Boots  
Ability: Flame Body  
Tera Type: Water  
EVs: 252 SpA / 4 SpD / 252 Spe  
Timid Nature  
IVs: 0 Atk  
- Quiver Dance  
- Bug Buzz  
- Flamethrower  
- Tera Blast  

Hatterene (F) @ Leftovers  
Ability: Magic Bounce  
Tera Type: Water  
EVs: 252 HP / 200 Def / 56 Spe  
Bold Nature  
IVs: 0 Atk  
- Calm Mind  
- Draining Kiss  
- Psyshock  
- Mystical Fire  

Roaring Moon @ Booster Energy  
Ability: Protosynthesis  
Tera Type: Flying  
EVs: 252 Atk / 4 SpD / 252 Spe  
Jolly Nature  
- Dragon Dance  
- Knock Off  
- Acrobatics  
- Earthquake  

Glimmora @ Focus Sash  
Ability: Toxic Debris  
Tera Type: Ghost  
EVs: 4 Def / 252 SpA / 252 Spe  
Timid Nature  
- Stealth Rock  
- Mortal Spin  
- Earth Power  
- Power Gem  

Kingambit @ Leftovers  
Ability: Supreme Overlord  
Tera Type: Fighting  
EVs: 208 HP / 252 Atk / 48 Spe  
Adamant Nature  
- Swords Dance  
- Low Kick  
- Sucker Punch  
- Iron Head  

Iron Boulder @ Booster Energy  
Ability: Quark Drive  
Tera Type: Fighting  
EVs: 252 Atk / 4 SpD / 252 Spe  
Jolly Nature  
- Zen Headbutt  
- Close Combat  
- Mighty Cleave  
- Swords Dance  
""" #HO Team by Mimikyu Stardust

HStall = """Hydrapple @ Heavy-Duty Boots  
Ability: Sticky Hold  
Tera Type: Ice  
EVs: 244 HP / 252 Def / 12 SpD  
Bold Nature  
IVs: 0 Atk / 27 Spe  
- Recover  
- Body Press  
- Giga Drain  
- Tera Blast  

Blissey @ Heavy-Duty Boots  
Ability: Natural Cure  
Tera Type: Water  
EVs: 4 HP / 252 Def / 252 SpD  
Calm Nature  
IVs: 0 Atk  
- Soft-Boiled  
- Calm Mind  
- Stealth Rock  
- Seismic Toss  

Dondozo @ Heavy-Duty Boots  
Ability: Unaware  
Tera Type: Fighting  
EVs: 252 HP / 252 Def / 4 SpD  
Impish Nature  
- Waterfall  
- Rest  
- Sleep Talk  
- Curse  

Clodsire @ Heavy-Duty Boots  
Ability: Unaware  
Tera Type: Steel  
EVs: 116 HP / 148 Def / 244 SpD  
Careful Nature  
- Recover  
- Amnesia  
- Poison Sting  
- Bulldoze  

Alomomola @ Heavy-Duty Boots  
Ability: Regenerator  
Tera Type: Dark  
EVs: 12 HP / 252 Def / 244 SpD  
Relaxed Nature  
IVs: 0 Spe  
- Wish  
- Protect  
- Scald  
- Flip Turn  

Gliscor @ Toxic Orb  
Ability: Poison Heal  
Tera Type: Ghost  
EVs: 244 HP / 252 Def / 12 SpD  
Impish Nature  
- Protect  
- Knock Off  
- Spikes  
- Poison Jab  
"""

# Initialize BARRY

In [15]:
MMKS = RandomPlayer(battle_format="gen9ou", team=HStall)
env = Barry(
    battle_format="gen9ou", opponent=MMKS, start_challenging=True,  team =SUSHO 
)
#check_env(barry)
#barry.close()

## Memory

In [16]:
class Memory:
    def __init__(self, len):
        self.is_done = deque(maxlen=len)
        self.action = deque(maxlen=len)
        self.rewards = deque(maxlen=len)
        self.state = deque(maxlen=len)

    def update(self, state, action, reward, done):
        # if the episode is finished we do not save to new state. Otherwise we have more states per episode than rewards
        # and actions whcih leads to a mismatch when we sample from memory.
        if not done:
            self.state.append(state)
        self.action.append(action)
        self.rewards.append(reward)
        self.is_done.append(done)

    def sample(self, batch_size):
        """
        sample "batch_size" many (state, action, reward, next state, is_done) datapoints.
        """
        n = len(self.is_done)
        idx = random.sample(range(0, n-1), batch_size)

        return torch.Tensor(self.state)[idx].to(device), torch.LongTensor(self.action)[idx].to(device), \
               torch.Tensor(self.state)[1+np.array(idx)].to(device), torch.Tensor(self.rewards)[idx].to(device), \
               torch.Tensor(self.is_done)[idx].to(device)

    def reset(self):
        self.rewards.clear()
        self.state.clear()
        self.action.clear()
        self.is_done.clear()

Select Action

In [17]:
def select_action(model, env, state, eps):
    state = torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(device)
    with torch.no_grad():
        values = model(state)

    # select a random action wih probability eps
    if random.random() <= eps:
        action = np.random.randint(0, env.action_space.n)
    else:
        action = np.argmax(values.cpu().numpy())

    return action

Train

In [18]:
def train(batch_size, current, target, optim, memory, gamma):

    states, actions, next_states, rewards, is_done = memory.sample(batch_size)

    q_values = current(states)

    next_q_values = current(next_states)
    next_q_state_values = target(next_states)

    q_value = q_values.gather(1, actions.unsqueeze(1)).squeeze(1)
    next_q_value = next_q_state_values.gather(1, torch.max(next_q_values, 1)[1].unsqueeze(1)).squeeze(1)
    expected_q_value = rewards + gamma * next_q_value * (1 - is_done)

    loss = (q_value - expected_q_value.detach()).pow(2).mean()

    optim.zero_grad()
    loss.backward()
    #Gradient Cliping?
    optim.step()


In [ ]:
def evaluate(Qmodel, env, repeats):
    """
    Runs a greedy policy with respect to the current Q-Network for "repeats" many episodes. Returns the average
    episode reward.
    """
    Qmodel.eval()
    perform = 0
    for _ in range(repeats):
        state, info = env.reset()
        done = False
        while not done:
            state = torch.Tensor(state).to(device)
            with torch.no_grad():
                values = Qmodel(state)
            action = np.argmax(values.cpu().numpy())
            state, reward, done, truncated, _ = env.step(action)
            perform += reward
    Qmodel.train()
    return perform/repeats


def update_parameters(current_model, target_model):
    target_model.load_state_dict(current_model.state_dict())

2024-04-21 20:30:12,353 - RandomPlayer 2 - ERROR - Unhandled exception raised while handling message:
>battle-gen9ou-2730
|init|battle
|title|Barry 2 vs. RandomPlayer 2
|j|☆Barry 2
|j|☆RandomPlayer 2
|t:|1713749120
|gametype|singles
|player|p1|Barry 2|101|
|player|p2|RandomPlayer 2|102|
|teamsize|p1|6
|teamsize|p2|6
|gen|9
|tier|[Gen 9] OU
|rule|Species Clause: Limit one of each Pokémon
|rule|OHKO Clause: OHKO moves are banned
|rule|Evasion Items Clause: Evasion items are banned
|rule|Evasion Moves Clause: Evasion moves are banned
|rule|Endless Battle Clause: Forcing endless battles is banned
|rule|HP Percentage Mod: HP is shown in percentages
|rule|Sleep Moves Clause: Sleep-inducing moves are banned
|clearpoke
|poke|p1|Volcarona, F|
|poke|p1|Hatterene, F|
|poke|p1|Roaring Moon|
|poke|p1|Glimmora, M|
|poke|p1|Kingambit, M|
|poke|p1|Iron Boulder|
|poke|p2|Hydrapple, F|
|poke|p2|Blissey, F|
|poke|p2|Dondozo, M|
|poke|p2|Clodsire, F|
|poke|p2|Alomomola, M|
|poke|p2|Gliscor, F|
|teamprevie

In [20]:
print(battle_tester)

None


# MAIN

In [26]:

#Parameters
batch_size = 64
gamma = 0.99
lr_gamma=0.9
lr = 0.0001
lr_step = 100
horizon = 150
num_episodes = 500
min_episodes = 5
measure_step = 10
measure_repeats = 10
eps = 1
eps_decay = 0.995
eps_min = 0.1
update_step = 10
max_memory_size = 10000

state, info = env.reset()
n_observations = len(state)
n_actions = env.action_space.n
#torch.manual_seed(seed)
#env.seed(seed)

Q_1 = DQN(n_observations, n_actions).to(device)
Q_2 = DQN(n_observations, n_actions).to(device)
# transfer parameters from Q_1 to Q_2
update_parameters(Q_1, Q_2)

# we only train Q_1
for param in Q_2.parameters():
    param.requires_grad = False

optimizer = torch.optim.Adam(Q_1.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=lr_step, gamma=lr_gamma)

memory = Memory(max_memory_size)
performance = []

for episode in range(num_episodes):
# display the performance
    if (episode % measure_step == 0) and episode >= min_episodes:
        performance.append([episode, evaluate(Q_1, env, measure_repeats)])
        print("Episode: ", episode)
        print("rewards: ", performance[-1][1])
        print("lr: ", scheduler.get_lr()[0])
        print("eps: ", eps)

    state, info = env.reset()
    
    memory.state.append(state)

    done = False
    i = 0
    while not done:
        i += 1
        action = select_action(Q_2, env, state, eps)
        state, reward, done, truncated, _ = env.step(action) #Truncated?

        if i > horizon:
            done = True

        # save state, action, reward sequence
        memory.update(state, action, reward, done)

    if episode >= min_episodes and episode % update_step == 0:
        for _ in range(50):
            train(batch_size, Q_1, Q_2, optimizer, memory, lr_gamma)

        # transfer new parameter from Q_1 to Q_2
        update_parameters(Q_1, Q_2)

    # update learning rate and eps
    scheduler.step()
    eps = max(eps*eps_decay, eps_min)

    print(performance)

Game Lost
[]


c:\Users\Francis Ng\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\optim\lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Game Won
[]
Game Won
[]
Game Won
[]
Game Lost
[]
Game Won
[]
Game Lost
[]
Game Won
[]
Game Won
[]
Game Won
[]
Game Won
Game Lost
Game Won
Game Lost
Game Won
Game Won
Game Won
Game Lost
Game Won
Game Won
Episode:  10
rewards:  13.43261652509897
lr:  0.0001
eps:  0.9511101304657719


c:\Users\Francis Ng\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\optim\lr_scheduler.py:389: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Game Won


C:\Users\Francis Ng\AppData\Local\Temp\ipykernel_9328\563276744.py:24: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:248.)
  return torch.Tensor(self.state)[idx].to(device), torch.LongTensor(self.action)[idx].to(device), \


[[10, 13.43261652509897]]
Game Won
[[10, 13.43261652509897]]
Game Lost
[[10, 13.43261652509897]]
Game Lost
[[10, 13.43261652509897]]
Game Won
[[10, 13.43261652509897]]
Game Won
[[10, 13.43261652509897]]
Game Lost
[[10, 13.43261652509897]]
Game Won
[[10, 13.43261652509897]]
Game Lost
[[10, 13.43261652509897]]
Game Won
[[10, 13.43261652509897]]
Game Won
Game Won
Game Lost
Game Won
Game Lost
Game Won
Game Won
Game Won
Game Lost
Game Won
Episode:  20
rewards:  13.670355431370421
lr:  0.0001
eps:  0.9046104802746175
Game Won
[[10, 13.43261652509897], [20, 13.670355431370421]]
Game Won
[[10, 13.43261652509897], [20, 13.670355431370421]]
Game Lost
[[10, 13.43261652509897], [20, 13.670355431370421]]
Game Won
[[10, 13.43261652509897], [20, 13.670355431370421]]
Game Won
[[10, 13.43261652509897], [20, 13.670355431370421]]
Game Lost
[[10, 13.43261652509897], [20, 13.670355431370421]]
Game Lost
[[10, 13.43261652509897], [20, 13.670355431370421]]
Game Won
[[10, 13.43261652509897], [20, 13.6703554313

In [21]:
state, info = env.reset()

In [20]:
Q_1 = DQN(n_observations, n_actions).to(device)
Q_2 = DQN(n_observations, n_actions).to(device)

In [17]:
battle1 = []
#Active Field
battle1 += [0]*14
battle1 += [0, 0, 0, 0, 0, 100, 1, 0]*6
battle1 += battle1
print(len(battle1))

124


In [29]:
n_observations = len(state)
n_actions = env.action_space.n
eps = 1

In [26]:
memory = Memory(max_memory_size)

In [25]:
action = select_action(Q_2, env, state, eps)
state, reward, done, truncated, _ = env.step(action) #Truncated?

In [88]:
# for p in list(battle_tester._teampreview_opponent_team):
#     if ("p2: " + p.species.capitalize()) in battle_tester.opponent_team:
#         print("It's in")
#     else:
#         print("It's not in")
battle_tester.opponent_team["p2: Hydrapple"]

KeyError: 'p2: Hydrapple'

In [85]:
battle_tester.opponent_team

{'p2: Gliscor': gliscor (pokemon object) [Active: True, Status: None]}

In [24]:
#Parameters
batch_size = 64
gamma = 0.99
lr_gamma=0.9
lr = 0.0001
lr_step = 100
horizon = 150
num_episodes = 500
min_episodes = 5
measure_step = 10
measure_repeats = 10
eps = 1
eps_decay = 0.995
eps_min = 0.1
update_step = 10
max_memory_size = 10000

state, info = env.reset()
n_observations = len(state)
n_actions = env.action_space.n
#torch.manual_seed(seed)
#env.seed(seed)

Q_1 = DQN(n_observations, n_actions).to(device)
Q_2 = DQN(n_observations, n_actions).to(device)
# transfer parameters from Q_1 to Q_2
update_parameters(Q_1, Q_2)

# we only train Q_1
for param in Q_2.parameters():
    param.requires_grad = False

optimizer = torch.optim.Adam(Q_1.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=lr_step, gamma=lr_gamma)

memory = Memory(max_memory_size)
performance = []

2024-04-21 20:31:50,110 - Barry 2 - ERROR - Unhandled exception raised while handling message:
>battle-gen9ou-2730
|init|battle
|title|Barry 2 vs. RandomPlayer 2
|j|☆Barry 2
|j|☆RandomPlayer 2
|t:|1713749120
|gametype|singles
|player|p1|Barry 2|101|
|player|p2|RandomPlayer 2|102|
|teamsize|p1|6
|teamsize|p2|6
|gen|9
|tier|[Gen 9] OU
|rule|Species Clause: Limit one of each Pokémon
|rule|OHKO Clause: OHKO moves are banned
|rule|Evasion Items Clause: Evasion items are banned
|rule|Evasion Moves Clause: Evasion moves are banned
|rule|Endless Battle Clause: Forcing endless battles is banned
|rule|HP Percentage Mod: HP is shown in percentages
|rule|Sleep Moves Clause: Sleep-inducing moves are banned
|clearpoke
|poke|p1|Volcarona, F|
|poke|p1|Hatterene, F|
|poke|p1|Roaring Moon|
|poke|p1|Glimmora, M|
|poke|p1|Kingambit, M|
|poke|p1|Iron Boulder|
|poke|p2|Hydrapple, F|
|poke|p2|Blissey, F|
|poke|p2|Dondozo, M|
|poke|p2|Clodsire, F|
|poke|p2|Alomomola, M|
|poke|p2|Gliscor, F|
|teampreview
|
|t:

KeyboardInterrupt: 